In [91]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
#Sklearn imports(and othrs)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.datasets import make_classification 
from datetime import datetime

In [166]:
#Load relevant data and create the full dataframe


filepath_met = "./data/Metropolitan-street.parquet"

# read the Parquet file into a pandas dataframe
df = pq.read_pandas(filepath_met).to_pandas()


barnet_df = df[df['LSOA name'].str.contains('Barnet')==True]
barnetBur_df = barnet_df[barnet_df['Crime type'].str.contains('Burglary')==True]
#barnetBur_df.head()

wards_csv = pd.read_csv("./data/LSOAtoWard.csv")
wards_relevant = wards_csv[['LSOA01NM', 'WD11NM']]
#wards_relevant

#Merge the main dataframe with the wards dataframe
merged_df = pd.merge(left=barnetBur_df, right=wards_relevant, left_on='LSOA name', right_on='LSOA01NM')

#Count the crimes comitted in a ward and the crimes in previous time windows in the same ward
df_crimes = merged_df[["Month", "Crime ID","WD11NM"]].copy()
df_crimes= df_crimes.groupby(["WD11NM","Month"]).count()

In [167]:

def rolling_sum2(group,y,str,s):
    group[str] = group['Crime ID'].rolling(window=y).sum().shift(s)
    return group

df_crimes = df_crimes.groupby('WD11NM', as_index=False).apply(lambda x: rolling_sum2(x,1,"1Months",1))
df_crimes.fillna(0, inplace=True)
df_crimes = df_crimes.groupby('WD11NM', as_index=False).apply(lambda x: rolling_sum2(x,2,"2Months",1))
df_crimes.fillna(0, inplace=True)
df_crimes = df_crimes.groupby('WD11NM', as_index=False).apply(lambda x: rolling_sum2(x,3,"3Months",1))
df_crimes.fillna(0, inplace=True)
df_crimes = df_crimes.groupby('WD11NM', as_index=False).apply(lambda x: rolling_sum2(x,4,"4Months",1))
df_crimes.fillna(0, inplace=True)
df_crimes = df_crimes.groupby('WD11NM', as_index=False).apply(lambda x: rolling_sum2(x,12,"12Months",1))
df_crimes.fillna(0, inplace=True)

#df_crimes = df_crimes.groupby('WD11NM', as_index=False).apply(lambda x: rolling_sum2(x,1,"1MonthsLag",2))
df_crimes.fillna(0, inplace=True)
df_crimes = df_crimes.groupby('WD11NM', as_index=False).apply(lambda x: rolling_sum2(x,2,"2MonthsLag",3))
df_crimes.fillna(0, inplace=True)
#df_crimes = df_crimes.groupby('WD11NM', as_index=False).apply(lambda x: rolling_sum2(x,3,"3MonthsLag",4))
df_crimes.fillna(0, inplace=True)
df_crimes = df_crimes.groupby('WD11NM', as_index=False).apply(lambda x: rolling_sum2(x,4,"4MonthsLag",5))
df_crimes.fillna(0, inplace=True)

#Merge the count of crimes to the main dataframe
merged_df = pd.merge(merged_df, df_crimes, on=['Month', 'WD11NM'], how='left')

merged_df.head()

C:\Users\20211445\AppData\Local\Temp\ipykernel_11300\1711153486.py:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_crimes = df_crimes.groupby('WD11NM', as_index=False).apply(lambda x: rolling_sum2(x,1,"1Months",1))
C:\Users\20211445\AppData\Local\Temp\ipykernel_11300\1711153486.py:7: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 


,Crime ID_x,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,...,LSOA01NM,WD11NM,Crime ID_y,1Months,2Months,3Months,4Months,12Months,2MonthsLag,4MonthsLag
0,48d7fae1e76d275cfecabf17efbf6fecf769966089dcae...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.205163,51.655067,On or near Salisbury Road,E01000248,Barnet 001A,Burglary,...,Barnet 001A,High Barnet,7,3.0,15.0,24.0,36.0,161.0,21.0,59.0
1,d1d37e3e233d650abaf3eae1d6585e33f706cebe857ffc...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.203868,51.653824,On or near Union Street,E01000248,Barnet 001A,Burglary,...,Barnet 001A,High Barnet,7,3.0,15.0,24.0,36.0,161.0,21.0,59.0
2,6b13c905e40eccf73feec600e87a6c70e0667b10980e95...,2020-07,Metropolitan Police Service,Metropolitan Police Service,-0.207839,51.654289,On or near The Avenue,E01000248,Barnet 001A,Burglary,...,Barnet 001A,High Barnet,6,10.0,17.0,20.0,32.0,137.0,15.0,48.0
3,9869e99b7803ce398498332b862347626619671258ecdb...,2021-01,Metropolitan Police Service,Metropolitan Police Service,-0.205204,51.656578,On or near Strafford Road,E01000248,Barnet 001A,Burglary,...,Barnet 001A,High Barnet,10,4.0,16.0,26.0,44.0,118.0,28.0,38.0
4,83c73a8dfb5749e2f3922ff87ae87e3f0422e3cd8c88fa...,2021-01,Metropolitan Police Service,Metropolitan Police Service,-0.203575,51.655735,On or near Shopping Area,E01000248,Barnet 001A,Burglary,...,Barnet 001A,High Barnet,10,4.0,16.0,26.0,44.0,118.0,28.0,38.0


In [168]:

#Encode the wards
encoded_wards = pd.get_dummies(merged_df['WD11NM'])
df_months = pd.get_dummies(merged_df['Month'].str.split("-").str[-1].astype(int))
df_months.columns = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'okt', 'nov', 'dec']
#Creating the final dataframe with only relevant columns for the linear regression

df_final = encoded_wards
#Add months
df_final = df_final.join(df_months)
df_final['1Months'] = merged_df['1Months']
#df_final['3Months'] = merged_df['3Months']
df_final['2Months'] = merged_df['2Months']
#df_final['2MonthsLag'] = merged_df['2MonthsLag']
#df_final['4Months'] = merged_df['4Months']
df_final['4MonthsLag'] = merged_df['4MonthsLag']
#df_final['12Months'] = merged_df['12Months']
df_final['Crime ID_y'] = merged_df['Crime ID_y']




#Drop NaN otherwise linear regression doesnt work(Find out where the NaN is!!!)
df_final = df_final.dropna()

#Get the labels and features of the data
y = df_final['Crime ID_y']
X = df_final.drop(['Crime ID_y'], axis=1)
df_final


,Brunswick Park,Burnt Oak,Childs Hill,Colindale,Coppetts,East Barnet,East Finchley,Edgware,Finchley Church End,Garden Suburb,...,jul,aug,sep,okt,nov,dec,1Months,2Months,4MonthsLag,Crime ID_y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3.0,15.0,59.0,7
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3.0,15.0,59.0,7
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,10.0,17.0,48.0,6
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4.0,16.0,38.0,10
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4.0,16.0,38.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43637,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,15.0,22.0,55.0,8
43638,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,8.0,23.0,43.0,15
43639,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,15.0,23.0,32.0,16
43640,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,15.0,23.0,32.0,16


In [169]:

#Drop NaN otherwise linear regression doesnt work(Find out where the NaN is!!!)
df_final = df_final.dropna()
merged_df["Month"]=pd.to_datetime(merged_df["Month"])
merged_df["Year"]=merged_df["Month"].dt.year
df_final["Year"]=merged_df["Year"]
#Get the labels and features of the data
y = df_final['Crime ID_y']
X = df_final.drop(['Crime ID_y'], axis=1)
df_final



,Brunswick Park,Burnt Oak,Childs Hill,Colindale,Coppetts,East Barnet,East Finchley,Edgware,Finchley Church End,Garden Suburb,...,aug,sep,okt,nov,dec,1Months,2Months,4MonthsLag,Crime ID_y,Year
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3.0,15.0,59.0,7,2020
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3.0,15.0,59.0,7,2020
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,10.0,17.0,48.0,6,2020
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4.0,16.0,38.0,10,2021
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4.0,16.0,38.0,10,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43637,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,15.0,22.0,55.0,8,2019
43638,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,8.0,23.0,43.0,15,2019
43639,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,15.0,23.0,32.0,16,2020
43640,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,15.0,23.0,32.0,16,2020


In [170]:



#Linera regression for the crimes in a ward
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,
                                               random_state=42, shuffle=True)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# calculate R-squared for the predictions
r2 = r2_score(y_test, y_pred)
print("R-squared: ", r2)
mse = mean_squared_error(y_test, y_pred)
print("Mean squared error: ", mse)

coefficients = model.coef_
coefficients
y_pred[:20]


R-squared:  0.5107531362840677
Mean squared error:  36.10863576061714


array([15.01812744, 23.05407715, 21.07720947, 15.8416748 ,  2.43151855,
        8.20214844, 14.79394531, 17.10906982, 16.22576904, 11.0880127 ,
       14.35467529,  9.62664795, 14.80517578, 23.21984863, 18.67974854,
       12.67132568, 11.04223633, 11.7409668 , 14.02368164, 19.16815186])

In [164]:
y_test[:20]

C:\Users\20211445\AppData\Local\Temp\ipykernel_11300\2616895430.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test[:20]


22081    21
13958    29
13754    19
43283    17
41208     0
873       7
22311    21
15984    24
40405    19
34460     4
18789    12
28512    10
22231    11
36825    22
25593    11
24592    10
8668     13
40019    17
36691    12
7836     26
Name: Crime ID_y, dtype: int64

In [171]:
df_predictors = merged_df[["1Months" , "4MonthsLag","Year", "Crime ID_y"]]
#"2Months"
#"4Months"
# compute the correlation matrix
corr_matrix = df_predictors.corr()
print(corr_matrix)

             1Months  4MonthsLag      Year  Crime ID_y
1Months     1.000000    0.551160  0.065721    0.653660
4MonthsLag  0.551160    1.000000  0.204521    0.524606
Year        0.065721    0.204521  1.000000    0.045109
Crime ID_y  0.653660    0.524606  0.045109    1.000000


In [ ]:



# plot the predicted values against the actual values
plt.scatter(y_pred, y_test,s=3)
plt.xlabel("Predicted values")
plt.ylabel("Actual values")
plt.title("Linear regression results")
plt.show()